# Four Traffic Management Models

for paper 'Traffic management and resource allocation for UAV-based parcel delivery in low-altitude urban space'

https://doi.org/10.1016/j.trc.2022.103808

In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt 
import networkx as nx
import pandas as pd
import warnings 
warnings.filterwarnings("ignore")
import community
import networkx.algorithms.community as nx_comm
from docplex.mp.model import Model
from networkx.algorithms.community.centrality import girvan_newman
import itertools
import time

In [42]:
#predetermined spatial conflict pairs of drones
Conflict=pd.read_csv('conflict_detection_results.csv')
#generated OD
ODsimulation=pd.read_csv('ODsimulation.csv')



edata=Conflict.groupby(['fn',"f'n"]).sum().reset_index()[['fn',"f'n",'long_s']]

In [47]:
# configurations
timeperiod=300 # time period length in sec
N=200  # number of flights

flighto=np.array(range(N))
path=np.array([0,1])
#piecewise delay cost coefficents
r1 = pd.DataFrame(np.array(range(1,N+1))/N*2,columns=['r1o'])
r2 = pd.DataFrame(np.array(range(1,N+1))*4/N,columns=['r2o'])

np.random.seed(123)
# scheduled departure time in unit sec
st = pd.DataFrame(np.random.uniform(low=0, high=timeperiod, size=(N,)),columns=['sto']) 
# path cost parameter
costo = pd.DataFrame(np.stack((np.array(ODsimulation['totalcost1']),np.array(ODsimulation['totalcost2']))).T,columns=['optimal', 'second'])
M=1000000 #big M
t0=10 #safety separation time

# Sequential Delay (SD) Model

In [48]:
start_time = time.time()

#input con:predetermined spatial conflict pairs
#      delaynow: current flight delay information 
def checkcon(con,delaynow):
    while True:
        con['test0']=con["ptbf'"]-con["ptef"]-sts[order[i]]-10 #triggered
        con['test1']=con["ptef'"]+10-con["ptbf"]-sts[order[i]] #delay
        con['free']=con["ptbf"]+sts[order[i]]-con["ptef'"]-10 
        valid=con[con["f'"].isin(list(list(order[:i])))].reset_index(drop=True)
        if not valid.empty:
            valid['tassigned']=valid["f'"].apply(lambda x: results_b[results_b['f']==x]['t'].values[0])
            valid=valid[valid['free']<valid['tassigned']]
            triggered=valid[valid['test0']+valid['tassigned']>=0]
            triggered['maxtime']=triggered["ptbf'"]+triggered['tassigned']-triggered["ptef"]-sts2[order[i]]-10
            valid=valid[valid['test0']+valid['tassigned']<0]
            ccc=valid[valid['test1']+valid['tassigned']>0]
            if not ccc.empty:
                ccc['delay']=ccc["ptef'"]+ccc['tassigned']-ccc["ptbf"]-sts2[order[i]]+10
                delaynow=np.max(ccc['delay'])
            if not triggered.empty:
                if delaynow<=np.min(triggered['maxtime']):
                    cur_results={'f':order[i],"t":sts2[order[i]]+delaynow,'lastdelay':delaynow}
                    return cur_results
                else:
                    sts[order[i]]=sts2[order[i]]+delaynow
            else:
                cur_results={'f':order[i],"t":sts2[order[i]]+delaynow,'lastdelay':delaynow}
                return cur_results
        else:
            cur_results={'f':order[i],"t":sts2[order[i]],'lastdelay':delaynow}
            return cur_results

data=Conflict[(Conflict['p']==0)&(Conflict["p'"]==0)].reset_index(drop=True)
nn=200
sts2=np.round(st['sto'],5)
sts=np.round(st['sto'],5)
r1s=r1['r1o']
r2s=r2['r2o']
np.random.seed(123)
order=np.random.permutation(nn)

results_b=pd.DataFrame(columns=["f","t",'lastdelay'])
results_b=results_b.append({'f':order[0],"t":sts[order[0]],'lastdelay':0},ignore_index=True)
for i in range(1, len(order)):

    con1=data[(data['f']==order[i])]
    con=pd.DataFrame(columns=["f","f'","ptbf","ptbf'","ptef","ptef'","p","p'"])
    con=con.append(con1.iloc[:,1:]).reset_index(drop=True)
    con2=data[(data["f'"]==order[i])]
    con2.loc[:,["f","f'","ptbf","ptbf'","ptef","ptef'","p","p'"]] = con2.loc[:,["f'","f","ptbf'","ptbf","ptef'","ptef","p'","p"]].values
    con=con.append(con2.iloc[:,1:]).reset_index(drop=True)
    delaynow=0
    if not con.empty:
        results_b=results_b.append(checkcon(con,delaynow),ignore_index=True)
    else:
        results_b=results_b.append({'f':order[i],"t":sts[order[i]],'lastdelay':0},ignore_index=True)   

results_b['delay']=results_b['t']-np.array(sts2[list(map(int, results_b['f']))])
results_b['r1']=np.array(r1s[list(map(int, results_b['f']))])
results_b['r2']=np.array(r2s[list(map(int, results_b['f']))])
results_b['cost']=0.15/60*(results_b['delay']*r1s+np.maximum((results_b['delay']-t0)*(r2s-r1s),np.zeros(200)))
print("--- %s seconds ---" % (time.time() - start_time))

print('total cost is: ', np.sum(results_b['cost'])+np.sum(costo.iloc[:200,0]))
print('Delay is: ', np.sum(results_b['delay']))
print('# delayed flights is: ', np.sum(results_b['delay']>=0.001))

--- 4.387779235839844 seconds ---
total cost is:  272.24538771532315
Delay is:  12316.478327912559
# delayed flights is:  65


# Sequential Delay & Reroute (SDR) Model

In [13]:
# configurations

# num of flights
nn=200 
# scheduled departure time
sts2=np.round(st['sto'],5) 
sts=np.round(st['sto'],5) 
# delay coeffcients
r1s=r1['r1o']   
r2s=r2['r2o']
data=Conflict
np.random.seed(123)
order=np.random.permutation(nn)

results=pd.DataFrame(columns=["f","t","path","cost"])
results=results.append({'f':order[0],"t":sts[order[0]],'path':0,'cost':costo.iloc[order[0],0]},ignore_index=True)

def seq(con, pathnow, costnow, delaynow):
    while True:
        triggered=pd.DataFrame(columns=["f'","p","pcost",'maxtime'])
        cost=pd.DataFrame(columns=["realdelay", "f'", 'p','pcost'])
        
        for j in range(i):
            a=con[(con["f'"]==order[j]) & (con["p'"]==results['path'][j])]
            if not a.empty:
                for k in a.index:
                    #if a["tbf"][k]+sts[order[i]]>=a["tef'"][k]+results['t'][j]+1/6.0:
                        #cost=cost.append({"f'":results["f"][j],"p":a['p'][k],'delay':0,'pcost':costo.iloc[order[i],a['p']].values[0]},ignore_index=True)
                    if a["ptbf'"][k]+results['t'][j]>=a["ptef"][k]+sts[order[i]]+10:
                        #cost=cost.append({"f'":results["f"][j],"p":a['p'][k],'delay':0,'pcost':costo.iloc[order[i],a['p']].values[0]},ignore_index=True)
                        triggered=triggered.append({"f'":results["f"][j],"p":a['p'][k],'maxtime': a["ptbf'"][k]+results['t'][j]-a["ptef"][k]-sts2[order[i]]-10,'pcost':costo.iloc[order[i],int(a['p'])]},ignore_index=True)
                    elif a["ptbf"][k]+sts[order[i]]<a["ptef'"][k]+results['t'][j]+10:
                        cost=cost.append({"f'":results["f"][j],"p":a['p'][k],'realdelay':a["ptef'"][k]+results['t'][j]-a["ptbf"][k]-sts2[order[i]]+10,'pcost':costo.iloc[order[i],int(a['p'])]},ignore_index=True)
        
        if not cost.empty:
            #cost['realdelay']=cost['delay']
            cost['cost']=0.15/60*(cost['realdelay']*r1s[order[i]]+np.maximum((cost['realdelay']-t0)*(r2s[order[i]]-r1s[order[i]]),np.zeros(len(cost))))+cost['pcost']
            d=[]
            c=[]
            if not cost[cost['p']==0].empty:   
                d.append(np.max(cost[cost['p']==0]['realdelay']))
                c.append(cost.loc[cost[cost['p']==0]['realdelay'].index[np.argmax(cost[cost['p']==0]['realdelay'])],'cost'])
            else:
                d.append(delaynow)
                c.append(0.15/60*(delaynow*r1s[order[i]]+np.maximum((delaynow-t0)*(r2s[order[i]]-r1s[order[i]]),0))+costo.iloc[order[i],0])
            if not cost[cost['p']==1].empty:
                d.append(np.max(cost[cost['p']==1]['realdelay']))
                c.append(cost.loc[cost[cost['p']==1]['realdelay'].index[np.argmax(cost[cost['p']==1]['realdelay'])],'cost'])
            else:
                d.append(delaynow)
                c.append(0.15/60*(delaynow*r1s[order[i]]+np.maximum((delaynow-t0)*(r2s[order[i]]-r1s[order[i]]),0))+costo.iloc[order[i],1])

            pathnow=np.argmin(c)
            costnow=np.min(c)
            delaynow=d[pathnow]
            if delaynow > np.min(triggered[triggered['p']==pathnow]['maxtime']):
                sts[order[i]]=sts2[order[i]]+delaynow
            else:
                cur_results={'f':order[i],"t":sts2[order[i]]+delaynow,'path':pathnow,'cost':costnow}
                return cur_results
        else:
            cur_results={'f':order[i],"t":sts2[order[i]]+delaynow,'path':pathnow,'cost':costnow}
            return cur_results

start_time = time.time()
for i in range(1,len(order)):

    con1=data[(data['f']==order[i])]
    con=pd.DataFrame(columns=["f","f'","ptbf","ptbf'","ptef","ptef'","p","p'"])
    con=con.append(con1.iloc[:,1:]).reset_index(drop=True)
    con2=data[(data["f'"]==order[i])]
    con2.loc[:,["f","f'","ptbf","ptbf'","ptef","ptef'","p","p'"]] = con2.loc[:,["f'","f","ptbf'","ptbf","ptef'","ptef","p'","p"]].values
    con=con.append(con2.iloc[:,1:]).reset_index(drop=True)
    pathnow=0
    delaynow=0
    costnow=costo.iloc[order[i],0]
    cur_results=seq(con, pathnow, costnow, delaynow)
    results=results.append(cur_results,ignore_index=True)
print("--- %s seconds ---" % (time.time() - start_time))

--- 19.58997392654419 seconds ---


In [14]:
print('Obj: ', np.round(np.sum(results['cost']),3))
#print('delay cost: ', np.round(np.sum(results['cost'])-np.sum(results['t']-list(sts2[order]))*0.15/60,3))
print('total delay: ', np.round(np.sum(results['t']-list(sts2[order])),3))
print('# Delayed flights: ', np.sum(results['t']-list(sts2[order])>0.0001))
print('# Alterantive path: ', np.sum(results['path']>0.0001))

Obj:  233.801
total delay:  1568.748
# Delayed flights:  33
# Alterantive path:  40


# Batch Optimization Model (BO)

In [34]:
start_time = time.time()

#node specification
node_list = np.sort(np.unique(np.append(Conflict['f'],Conflict["f'"])).astype(int))
original_f=list(set(range(200))-set(node_list))
node=node_list

#create network
G = nx.Graph()
G.add_nodes_from(node)
edata2=Conflict.groupby(['fn',"f'n"]).sum().reset_index()[['fn',"f'n",'long_s']]
edge_list = list(edata2[['fn',"f'n",'long_s']].astype(int).itertuples(index=False, name=None))
G.add_weighted_edges_from(edge_list)

#clustering
partition = community.best_partition(G)
plist=[]
for com in set(partition.values()) :
    list_nodes = [nodes for nodes in partition.keys() if partition[nodes] == com]
    plist.append(list_nodes)

#group priority order
dc = nx.degree_centrality(G)
deg_cen=list(dc.values())
deg_group=[]
for i in range(len(plist)):
    deg_group.append(np.sum(list(map(dc.get, plist[i]))))
order=np.argsort(deg_group)[::-1]

#first run
flight=plist[order[0]]
N_now=len(flight)
cost=costo.iloc[flight,:]
conflict_pair = Conflict[(Conflict["f"].isin(flight)) & (Conflict["f'"].isin(flight))]
conflict2=conflict_pair[["f","f'",'p',"p'"]]
pair=tuple(map(tuple, np.array(conflict2)))
tef=list(np.round(np.array(conflict_pair["ptef"]),4))
tbf=list(np.round(np.array(conflict_pair["ptbf"]),4))
tefp=list(np.round(np.array(conflict_pair["ptef'"]),4))
tbfp=list(np.round(np.array(conflict_pair["ptbf'"]),4))

opl_r=[]
path0_r=[]
obj_r=[]
m = Model(name='full', log_output=True)
t = m.continuous_var_dict(flight, name="depart_time")
B = m.binary_var_dict(pair, name="f_first")
Bp = m.binary_var_dict(pair, name="fp_first")
I = m.binary_var_dict([(i,j) for i in flight for j in path], name="decision")
z = m.binary_var_dict(pair, name="conflict")
D = m.continuous_var_dict(flight, name="delay")
m.minimize(m.sum( cost.loc[i][j]*I[i,j] for i in flight for j in path) + 0.15/60*sum(D[i] for i in flight))
m.add_constraints( D[f] >= r1.loc[f][0]*(t[f]-st.loc[f][0]) for f in flight)
m.add_constraints( D[f] >= r2.loc[f][0]*(t[f]-st.loc[f][0])+r1.loc[f][0]*t0-r2.loc[f][0]*t0 for f in flight)
m.add_constraints( t[pair[k][0]]+tef[k]+10 <= M*(1-B[pair[k]])+tbfp[k]+t[pair[k][1]] for k in range(len(pair)))
m.add_constraints( t[pair[k][1]]+tefp[k]+10 <= M*(1-Bp[pair[k]])+tbf[k]+t[pair[k][0]] for k in range(len(pair)))
m.add_constraints( B[cc] + Bp[cc]==z[cc] for cc in pair)
m.add_constraints( z[cc] <= I[cc[0],cc[2]] for cc in pair)
m.add_constraints( z[cc] <= I[cc[1],cc[3]] for cc in pair)
m.add_constraints( z[cc] + 1 >= I[cc[0],cc[2]] + I[cc[1],cc[3]] for cc in pair)
m.add_constraints( t[f]>=st.loc[f][0] for f in flight)
m.add_constraints( m.sum( I[i,j] for j in path) ==1 for i in flight)
msol = m.solve()
assert msol is not None, "model can't solve"

t_opl=m.solution.get_values(list(t.values()))
b_path0=m.solution.get_values(list(I.values()))[0::2]
opl_r=np.append(opl_r,t_opl)
path0_r=np.append(path0_r, b_path0)
obj_r=np.append(obj_r, m.objective_value)

#later runs
for clu in range(1,len(order)):
    flight=plist[order[clu]]
    #N_now=len(flight)
    cost=costo.iloc[flight,:]
    conflict_pair = Conflict[(Conflict["f"].isin(flight)) & (Conflict["f'"].isin(flight))]
    conflict2=conflict_pair[["f","f'",'p',"p'"]].astype(int)
    pair=tuple(map(tuple, np.array(conflict2)))
    tef=list(np.round(np.array(conflict_pair["ptef"]),4))
    tbf=list(np.round(np.array(conflict_pair["ptbf"]),4))
    tefp=list(np.round(np.array(conflict_pair["ptef'"]),4))
    tbfp=list(np.round(np.array(conflict_pair["ptbf'"]),4))

    m = Model(name='full', log_output=True)
    t = m.continuous_var_dict(flight, name="depart_time")
    B = m.binary_var_dict(pair, name="f_first")
    Bp = m.binary_var_dict(pair, name="fp_first")
    I = m.binary_var_dict([(i,j) for i in flight for j in path], name="decision")
    z = m.binary_var_dict(pair, name="conflict")
    D = m.continuous_var_dict(flight, name="delay")
    m.minimize(m.sum( cost.loc[i][j]*I[i,j] for i in flight for j in path) + 0.15/60*sum(D[i] for i in flight))
    m.add_constraints( D[f] >= r1.loc[f][0]*(t[f]-st.loc[f][0]) for f in flight)
    m.add_constraints( D[f] >= r2.loc[f][0]*(t[f]-st.loc[f][0])+r1.loc[f][0]*t0-r2.loc[f][0]*t0 for f in flight)
    m.add_constraints( t[pair[k][0]]+tef[k]+10 <= M*(1-B[pair[k]])+tbfp[k]+t[pair[k][1]] for k in range(len(pair)))
    m.add_constraints( t[pair[k][1]]+tefp[k]+10 <= M*(1-Bp[pair[k]])+tbf[k]+t[pair[k][0]] for k in range(len(pair)))
    m.add_constraints( B[cc] + Bp[cc]==z[cc] for cc in pair)
    m.add_constraints( z[cc] <= I[cc[0],cc[2]] for cc in pair)
    m.add_constraints( z[cc] <= I[cc[1],cc[3]] for cc in pair)
    m.add_constraints( z[cc] + 1 >= I[cc[0],cc[2]] + I[cc[1],cc[3]] for cc in pair)
    m.add_constraints( t[f]>=st.loc[f][0] for f in flight)
    m.add_constraints( m.sum( I[i,j] for j in path) ==1 for i in flight)
    # deconflict with assigned
    assigned_f=np.hstack(np.array(plist)[order[:clu]])
    f_p0=assigned_f[np.array(path0_r)>0.01]
    f_p1=assigned_f[np.array(path0_r)<0.01]
    c_assign=pd.DataFrame(columns=["f","f'","ptbf","ptbf'","ptef","ptef'","p","p'"]) # separate 0 1 path
    #c_assign1=pd.DataFrame(columns=["f","f'","ptbf","ptbf'","ptef","ptef'","p","p'"]) 
    c1=Conflict[(Conflict["f"].isin(flight)) & (Conflict["p"]<0.01) & (Conflict["f'"].isin(f_p0)) & (Conflict["p'"]<0.01)]
    c_assign=c_assign.append(c1[["f","f'","ptbf","ptbf'","ptef","ptef'","p","p'"]]).reset_index(drop=True)
    c2=Conflict[(Conflict["f"].isin(flight)) & (Conflict["p"]<0.01) & (Conflict["f'"].isin(f_p1)) & (Conflict["p'"]>0.01)]
    c_assign=c_assign.append(c2[["f","f'","ptbf","ptbf'","ptef","ptef'","p","p'"]]).reset_index(drop=True)
    c3=Conflict[(Conflict["f'"].isin(flight)) & (Conflict["p"]<0.01) & (Conflict["f"].isin(f_p0)) & (Conflict["p"]<0.01)]
    c3.loc[:,["f","f'","ptbf","ptbf'","ptef","ptef'","p","p'"]] = c3.loc[:,["f'","f","ptbf'","ptbf","ptef'","ptef","p'","p"]].values
    c_assign=c_assign.append(c3[["f","f'","ptbf","ptbf'","ptef","ptef'","p","p'"]]).reset_index(drop=True)
    c4=Conflict[(Conflict["f'"].isin(flight)) & (Conflict["p"]<0.01) & (Conflict["f"].isin(f_p1)) & (Conflict["p"]>0.01)]
    c4.loc[:,["f","f'","ptbf","ptbf'","ptef","ptef'","p","p'"]] = c4.loc[:,["f'","f","ptbf'","ptbf","ptef'","ptef","p'","p"]].values
    c_assign=c_assign.append(c4[["f","f'","ptbf","ptbf'","ptef","ptef'","p","p'"]]).reset_index(drop=True)
    c_assign["t_f'_real"]=c_assign["f'"].apply(lambda x: opl_r[list(assigned_f).index(x)])
    B_assign = m.binary_var_dict(range(len(c_assign)), name="Assigned_first")
    m.add_constraints( t[c_assign.loc[qq]['f']]+c_assign.loc[qq]['ptef']+10 <= M*(1-B_assign[qq])+M*(1-I[c_assign.loc[qq]['f'],0])+c_assign.loc[qq]["ptbf'"]+c_assign.loc[qq]["t_f'_real"] for qq in range(len(c_assign)))
    m.add_constraints( c_assign.loc[qq]["ptef'"]+c_assign.loc[qq]["t_f'_real"]+10 <= M*B_assign[qq]+M*(1-I[c_assign.loc[qq]['f'],0])+t[c_assign.loc[qq]['f']]+c_assign.loc[qq]['ptbf'] for qq in range(len(c_assign)))
    
    c_assign1=pd.DataFrame(columns=["f","f'","ptbf","ptbf'","ptef","ptef'","p","p'"]) 
    c11=Conflict[(Conflict["f"].isin(flight)) & (Conflict["p"]>0.5) & (Conflict["f'"].isin(f_p0)) & (Conflict["p'"]<0.01)]
    c_assign1=c_assign1.append(c11[["f","f'","ptbf","ptbf'","ptef","ptef'","p","p'"]]).reset_index(drop=True)
    c21=Conflict[(Conflict["f"].isin(flight)) & (Conflict["p"]>0.5) & (Conflict["f'"].isin(f_p1)) & (Conflict["p'"]>0.01)]
    c_assign1=c_assign1.append(c21[["f","f'","ptbf","ptbf'","ptef","ptef'","p","p'"]]).reset_index(drop=True)
    c31=Conflict[(Conflict["f'"].isin(flight)) & (Conflict["p"]>0.5) & (Conflict["f"].isin(f_p0)) & (Conflict["p"]<0.01)]
    c31.loc[:,["f","f'","ptbf","ptbf'","ptef","ptef'","p","p'"]] = c31.loc[:,["f'","f","ptbf'","ptbf","ptef'","ptef","p'","p"]].values
    c_assign1=c_assign1.append(c31[["f","f'","ptbf","ptbf'","ptef","ptef'","p","p'"]]).reset_index(drop=True)
    c41=Conflict[(Conflict["f'"].isin(flight)) & (Conflict["p"]>0.5) & (Conflict["f"].isin(f_p1)) & (Conflict["p"]>0.01)]
    c41.loc[:,["f","f'","ptbf","ptbf'","ptef","ptef'","p","p'"]] = c41.loc[:,["f'","f","ptbf'","ptbf","ptef'","ptef","p'","p"]].values
    c_assign1=c_assign1.append(c41[["f","f'","ptbf","ptbf'","ptef","ptef'","p","p'"]]).reset_index(drop=True)
    c_assign1["t_f'_real"]=c_assign1["f'"].apply(lambda x: opl_r[list(assigned_f).index(x)])
    B_assign1 = m.binary_var_dict(range(len(c_assign1)), name="Assigned_first_1")
    m.add_constraints( t[c_assign1.loc[pp]['f']]+c_assign1.loc[pp]['ptef']+10 <= M*(1-B_assign1[pp])+M*(1-I[c_assign1.loc[pp]['f'],1])+c_assign1.loc[pp]["ptbf'"]+c_assign1.loc[pp]["t_f'_real"] for pp in range(len(c_assign1)))
    m.add_constraints( c_assign1.loc[pp]["ptef'"]+c_assign1.loc[pp]["t_f'_real"]+10 <= M*B_assign1[pp]+M*(1-I[c_assign1.loc[pp]['f'],1])+t[c_assign1.loc[pp]['f']]+c_assign1.loc[pp]['ptbf'] for pp in range(len(c_assign1)))

    msol = m.solve()
    assert msol is not None, "model can't solve"
    t_opl=m.solution.get_values(list(t.values()))
    b_path0=m.solution.get_values(list(I.values()))[0::2]
    opl_r=np.append(opl_r,t_opl)
    path0_r=np.append(path0_r, b_path0)
    obj_r=np.append(obj_r, m.objective_value)
obj_r2=np.sum(obj_r)+np.sum(costo.iloc[original_f,0])

allf=np.hstack(np.array(plist)[order])
hier_r=st.copy()[:200]
hier_r['ht']=-1
original_f=list(set(range(200))-set(node_list))
for i in original_f:
    hier_r.loc[i,'ht']=hier_r.loc[i,'sto']
for j in node_list:
    hier_r.loc[j,'ht']=opl_r[list(allf).index(j)]
hier_r['r1']=r1['r1o']
hier_r['r2']=r2['r2o']
hier_r['delay']=hier_r['ht']-hier_r['sto']

print("--- %s seconds ---" % (time.time() - start_time))
print('Obj hier : ', obj_r2)
print('total delay: ', np.round(np.sum(hier_r['ht']-hier_r['sto']),3))
print('# Delayed flights: ', np.sum(hier_r['delay']>0.0001))
print('# Alterantive path: ', np.sum(np.array(path0_r)<0.0001))


Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
Tried aggregator 4 times.
MIP Presolve eliminated 27 rows and 2 columns.
Aggregator did 27 substitutions.
Reduced MIP has 1564 rows, 830 columns, and 4136 nonzeros.
Reduced MIP has 780 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (4.65 ticks)
Probing time = 0.02 sec. (2.73 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 1564 rows, 830 columns, and 4136 nonzeros.
Reduced MIP has 780 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (3.09 ticks)
Probing time = 0.02 sec. (2.68 ticks)
Clique table members: 2909.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation solution time = 0.00 sec. (6.11 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer

Represolve time = 0.03 sec. (10.06 ticks)
Probing time = 0.02 sec. (0.81 ticks)
Clique table members: 695.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation solution time = 0.00 sec. (1.34 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                           10.2434       10.1759             0.66%
      0     0       10.2070    16       10.2434       10.2070      368    0.35%
      0     0       10.2366    16       10.2434      Cuts: 19      381    0.07%
      0     0        cutoff             10.2434                    383    0.00%
Elapsed time = 0.17 sec. (60.81 ticks, tree = 0.01 MB, solutions = 3)

Cover cuts applied:  1
Implied bound cuts applied:  19
Flow cuts applied:  3
Mixed integer rounding cuts applied:  2
Gomory fractional cuts applied:  6

Root node proc

MIP Presolve modified 1037 coefficients.
Aggregator did 22 substitutions.
Reduced MIP has 620 rows, 333 columns, and 1660 nonzeros.
Reduced MIP has 297 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (4.07 ticks)
Probing time = 0.02 sec. (2.21 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 2 rows and 0 columns.
MIP Presolve modified 4 coefficients.
Reduced MIP has 618 rows, 333 columns, and 1656 nonzeros.
Reduced MIP has 297 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.20 ticks)
Probing time = 0.00 sec. (2.16 ticks)
Clique table members: 1204.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation solution time = 0.00 sec. (1.86 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0           

Probing time = 0.00 sec. (1.51 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve modified 34 coefficients.
Reduced MIP has 506 rows, 264 columns, and 1422 nonzeros.
Reduced MIP has 248 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.00 ticks)
Probing time = 0.00 sec. (1.46 ticks)
Clique table members: 1011.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation solution time = 0.00 sec. (1.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                          355.1239        6.1708            98.26%
      0     0        6.2478     8      355.1239        6.2478      128   98.24%
*     0+    0                            8.4357        6.2478            25.94%
      0     0        6.2889     8        8.4357      Cuts: 57      1

      0     0        cutoff             10.2087       10.2086       49    0.00%
Elapsed time = 0.03 sec. (4.14 ticks, tree = 0.01 MB, solutions = 3)

Root node processing (before b&c):
  Real time             =    0.03 sec. (4.15 ticks)
Parallel b&c, 12 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.03 sec. (4.15 ticks)
--- 5.58821702003479 seconds ---
Obj hier :  159.66049136247582
total delay:  1643.477
# Delayed flights:  39
# Alterantive path:  58


# Full Optimization Model (FO)

In [16]:
start_time = time.time()

#configurations
N=200
flight=np.array(range(N))
cost=costo.iloc[flight,:]
conflict_pair = Conflict
conflict2=conflict_pair[["f","f'",'p',"p'"]].astype(int)
pair=tuple(map(tuple, np.array(conflict2)))
tef=list(np.round(np.array(conflict_pair["ptef"]),4))
tbf=list(np.round(np.array(conflict_pair["ptbf"]),4))
tefp=list(np.round(np.array(conflict_pair["ptef'"]),4))
tbfp=list(np.round(np.array(conflict_pair["ptbf'"]),4))


m = Model(name='full', log_output=True)

t = m.continuous_var_dict(flight, name="depart_time")
B = m.binary_var_dict(pair, name="f_first")
Bp = m.binary_var_dict(pair, name="fp_first")
I = m.binary_var_dict([(i,j) for i in flight for j in path], name="decision")
z = m.binary_var_dict(pair, name="conflict")
D = m.continuous_var_dict(flight, name="delay")

# objective function
m.minimize(m.sum( cost.loc[i][j]*I[i,j] for i in flight for j in path) + 0.15/60*sum(D[i] for i in flight))

# constraints
m.add_constraints( D[f] >= r1.loc[f][0]*(t[f]-st.loc[f][0]) for f in flight)
m.add_constraints( D[f] >= r2.loc[f][0]*(t[f]-st.loc[f][0])+r1.loc[f][0]*t0-r2.loc[f][0]*t0 for f in flight)
m.add_constraints( t[pair[k][0]]+tef[k]+10 <= M*(1-B[pair[k]])+tbfp[k]+t[pair[k][1]] for k in range(len(pair)))
m.add_constraints( t[pair[k][1]]+tefp[k]+10 <= M*(1-Bp[pair[k]])+tbf[k]+t[pair[k][0]] for k in range(len(pair)))
m.add_constraints( B[cc] + Bp[cc]==z[cc] for cc in pair)
m.add_constraints( z[cc] <= I[cc[0],cc[2]] for cc in pair)
m.add_constraints( z[cc] <= I[cc[1],cc[3]] for cc in pair)
m.add_constraints( z[cc] + 1 >= I[cc[0],cc[2]] + I[cc[1],cc[3]] for cc in pair)

m.add_constraints( t[f]>=st.loc[f][0] for f in flight)
m.add_constraints( m.sum( I[i,j] for j in path) ==1 for i in flight)

msol = m.solve()
assert msol is not None, "model can't solve"
print("--- %s seconds ---" % (time.time() - start_time))
print('Obj: ', np.round(m.objective_value,3))
print('total delay: ', np.round(np.sum(m.solution.get_values(list(t.values()))-st['sto'][:200]),3))
print('# Delayed flights: ', np.sum(m.solution.get_values(list(t.values()))-st['sto'][:200]>0.0001))
print('# Alterantive path: ', np.sum(np.array(m.solution.get_values(list(I.values()))[0::2])<0.0001))

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
Tried aggregator 4 times.
MIP Presolve eliminated 208 rows and 10 columns.
Aggregator did 200 substitutions.
Reduced MIP has 28916 rows, 14852 columns, and 76844 nonzeros.
Reduced MIP has 14456 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.11 sec. (94.83 ticks)
Probing time = 0.02 sec. (8.04 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 28916 rows, 14852 columns, and 76844 nonzeros.
Reduced MIP has 14456 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.08 sec. (67.45 ticks)
Probing time = 0.02 sec. (7.47 ticks)
Clique table members: 50948.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation solution time = 0.56 sec. (392.27 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective